<a href="https://colab.research.google.com/github/scudilio/FIAP/blob/main/RN_CNN_IMAGENS_COLORIDAS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Treinando uma CNN para Reconhecimento de Imagens do Homer e Bart
Este notebook demonstra o uso de uma rede neural convolucional (CNN) para reconhecer personagens do desenho "Os Simpsons", especificamente Homer e Bart. Usamos a biblioteca TensorFlow para criar e treinar o modelo, além de técnicas de pré-processamento e data augmentation para melhorar a precisão.


## Instalação de dependências e importação de **bibliotecas**

In [ ]:
# Instalando o TensorFlow versão 2.16.1
!pip install tensorflow==2.16.1 -q

Instala a versão específica do TensorFlow necessária para garantir compatibilidade com o código.

In [ ]:
#importando as bibliotecas

import numpy as np #biblioteca para trabalhar com arrays (vetores)
import tensorflow as tf #biblioteca para rede neurais (deep learning)


 As bibliotecas numpy e tensorflow são usadas para manipulação de dados e criação de redes neurais

In [ ]:
np.__version__, tf.__version__

('1.26.4', '2.16.1')

In [ ]:
#bibliotecas para trabalhar com arquivos .zip
import zipfile  #descompactar uma pasta zip
import tempfile # cria pastas temporarias

Vamos utilizar o zipfile e tempfile para trabalhar com arquivos compactados e utilizar uma pasta temporária.

In [ ]:
temp_dir = tempfile.TemporaryDirectory()
print(temp_dir)

<TemporaryDirectory '/tmp/tmpf8ns_8ua'>


Aqui, o dataset de personagens é descompactado para um diretório temporário. Isso facilita o uso dos dados sem necessidade de arquivos locais permanentes.

In [ ]:
# r read, leitura do arquivo

with zipfile.ZipFile('dataset_personagens.zip', 'r') as zip_ref:
  zip_ref.extractall(temp_dir.name)

Aqui, o dataset de personagens é descompactado para um diretório temporário. Isso facilita o uso dos dados sem necessidade de arquivos locais permanentes.

## Pré processamento da imagem

[Documentação da função para gerar novas imagens](https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/image/ImageDataGenerator)

In [ ]:
## biblioteca para gerar novas imagens a partir das fotos iniciais

from tensorflow.keras.preprocessing.image  import ImageDataGenerator

In [ ]:
# utilizar as imagens para gerar novas imagens

gerador_treinamento = ImageDataGenerator(rescale = 1/255, # normalizando os valores do pixels das imagens
                                         rotation_range = 7, #rotacao aleatoria da imagem
                                         horizontal_flip = True, #realiza a inversao horizontal, espelhamento da imagem
                                         shear_range = 0.2, #realiza o deslocamento da imagem criando distorcao
                                         height_shift_range = 0.07,#realiza deslocamento verticais
                                         zoom_range = 0.2 #aplica zoom
)

Utilizamos o ImageDataGenerator para aumentar o dataset aplicando transformações nas imagens, o que ajuda a generalizar o modelo

In [ ]:
gerador_teste = ImageDataGenerator(rescale = 1/255)

In [ ]:
base_treino = gerador_treinamento.flow_from_directory(f'{temp_dir.name}/dataset_personagens/training_set',
                                                target_size = (64,64),
                                                batch_size = 32,
                                                class_mode = 'binary'
                                                )

Found 196 images belonging to 2 classes.


In [ ]:
base_teste = gerador_teste.flow_from_directory(f'{temp_dir.name}/dataset_personagens/test_set',
                                                target_size = (64,64),
                                                batch_size = 32,
                                                class_mode = 'binary'
                                                )

Found 73 images belonging to 2 classes.


# Treinamento da Rede Neural Convolucional

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.layers import BatchNormalization, InputLayer

In [ ]:
cnn = Sequential()

In [ ]:
cnn.add(InputLayer(input_shape = (64,64,3)))
cnn.add(Conv2D(filters = 32, kernel_size = (3,3), activation = 'relu'))
cnn.add(BatchNormalization())
cnn.add(MaxPooling2D(pool_size = (2,2)))

cnn.add(Conv2D(filters = 32, kernel_size = (3,3), activation = 'relu'))
cnn.add(BatchNormalization())
cnn.add(MaxPooling2D(pool_size = (2,2)))

cnn.add(Flatten())

cnn.add(Dense(units = 128, activation = 'relu'))
cnn.add(Dropout(0.2))
cnn.add(Dense(units = 1, activation = 'sigmoid'))

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/input_layer.py:26: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


In [ ]:
cnn.summary()

Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d_6 (Conv2D)                    │ (None, 62, 62, 32)          │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_6                │ (None, 62, 62, 32)          │             128 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_6 (MaxPooling2D)       │ (None, 31, 31, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_7 (Conv2D)                    │ (None, 29, 29, 32)          │           9,248 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_7                │ (None, 29, 29, 32)          │             128 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_7 (MaxPooling2D)       │ (None, 14, 14, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_3 (Flatten)                  │ (None, 6272)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_6 (Dense)                      │ (None, 128)                 │         802,944 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_3 (Dropout)                  │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_7 (Dense)                      │ (None, 1)                   │             129 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 813,473 (3.10 MB)

 Trainable params: 813,345 (3.10 MB)

 Non-trainable params: 128 (512.00 B)

In [ ]:
cnn.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [ ]:
cnn.fit(base_treino, epochs = 50 , validation_data = base_teste )

Epoch 1/50
2/7 ━━━━━━━━━━━━━━━━━━━━ 1s 395ms/step - accuracy: 0.4609 - loss: 2.3420

/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


5/7 ━━━━━━━━━━━━━━━━━━━━ 1s 981ms/step - accuracy: 0.4780 - loss: 2.5454

/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


7/7 ━━━━━━━━━━━━━━━━━━━━ 12s 781ms/step - accuracy: 0.5095 - loss: 2.4048 - val_accuracy: 0.5753 - val_loss: 0.7299
Epoch 2/50
7/7 ━━━━━━━━━━━━━━━━━━━━ 2s 122ms/step - accuracy: 0.6898 - loss: 0.7802 - val_accuracy: 0.6849 - val_loss: 0.6624
Epoch 3/50
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step - accuracy: 0.7462 - loss: 0.6725 - val_accuracy: 0.4658 - val_loss: 0.6934
Epoch 4/50
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 31ms/step - accuracy: 0.7686 - loss: 0.4587 - val_accuracy: 0.4932 - val_loss: 0.6898
Epoch 5/50
7/7 ━━━━━━━━━━━━━━━━━━━━ 2s 20ms/step - accuracy: 0.7847 - loss: 0.3803 - val_accuracy: 0.4932 - val_loss: 0.6749
Epoch 6/50
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 144ms/step - accuracy: 0.8677 - loss: 0.3281 - val_accuracy: 0.4658 - val_loss: 0.6743
Epoch 7/50
7/7 ━━━━━━━━━━━━━━━━━━━━ 2s 118ms/step - accuracy: 0.8448 - loss: 0.3404 - val_accuracy: 0.7123 - val_loss: 0.5716
Epoch 8/50
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 35ms/step - accuracy: 0.8968 - loss: 0.2180 - val_accuracy: 0.6027 - val_loss: 0.6005
Epoch 

## Qual o percentual de acerto você conseguiu?

De acordo com o treinamento conseguimos uma acurácia de 97% e 80% na validação.

Epoch 50/50
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 25ms/step - accuracy: 0.9684 - loss: 0.0791 - val_accuracy: 0.8493 - val_loss: 0.5069

# Prevendo novas imagens

In [ ]:
from keras.preprocessing import image

In [ ]:
##criando uma funcao para classificar uma nova imagem

def predict_image(caminho):
  imagem_teste = image.load_img(caminho, target_size = (64,64)) #carregando imagem
  imagem_teste = image.img_to_array(imagem_teste) #transformando em array
  imagem_teste /= 255 # padronizando
  imagem_teste = np.expand_dims(imagem_teste, axis = 0) #definindo as dimensoes da imagem
  previsao = cnn.predict(imagem_teste)[0] #previsao

  return "Bart" if previsao < 0.5 else "Homer"

In [ ]:
predict_image("5.jpg")

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 919ms/step


'Homer'

## Conclusão

O modelo conseguiu prever corretamente a imagem do homer comerndo rosquinha